# MAGICC7

Pymagicc is also MAGICC7 compatible. 

As yet, the binary is not yet publicly available to it's a bit fiddlier to get setup. Here we give a minimal working example of how to use MAGICC7 with Pymagicc.

Temporary notes: 

- add reference period argument to line plot
- make seaborn plot to make style etc. nicer

In [ ]:
# NBVAL_IGNORE_OUTPUT

import pandas as pd
import pymagicc
from pymagicc import MAGICC6, MAGICC7
from pymagicc.io import MAGICCData

from pymagicc import rcp26, rcp45, rcp60, rcp85, scenarios

In [ ]:
# NBVAL_IGNORE_OUTPUT
%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 16, 9

In [ ]:
# set your environment to find your MAGICC binary
%env MAGICC_EXECUTABLE_7=../../magicc/run/magicc

## The slow, easy way

Use the `pymagicc.run` API. This is super slow as it makes a new copy of the entire `magicc/run` directory for each run.

A few common issues when using your own setup are:
    - output flag settings may mean that you don't get the output you expect
    - super slow running as lots of output are being read back in (turn `OUT_XX` flags off to speed things up)
    - super slow running as your `out` directory is not empty before you run
    - your config file has more than one emissions scenario and/or config flag set (Pymagicc deliberately throws an error here to avoid difficult debugging of overlapping emissions and config inputs)
    
`MAGCFG_USER.CFG` should have lines like

```FORTRAN90
  FILE_EMISSCEN =  "NONE",
  FILE_EMISSCEN_2 =  "",
  FILE_EMISSCEN_3 =  "",
  FILE_EMISSCEN_4 =  "",
  FILE_EMISSCEN_5 =  "",
  FILE_EMISSCEN_6 =  "",
  FILE_EMISSCEN_7 =  "",
  FILE_EMISSCEN_8 =  "",
  FILE_TUNINGMODEL_1 =  "PYMAGICC",
  FILE_TUNINGMODEL_2 =  "USER",
  FILE_TUNINGMODEL_3 =  "USER",
  FILE_TUNINGMODEL_4 =  "USER",
  FILE_TUNINGMODEL_5 =  "USER",
  FILE_TUNINGMODEL_6 =  "USER",
  FILE_TUNINGMODEL_7 =  "USER",
  FILE_TUNINGMODEL_8 =  "USER",
  FILE_TUNINGMODEL_9 =  "USER",
  FILE_TUNINGMODEL_10 =  "USER",
```

In [ ]:
for i, (name, scen) in enumerate(scenarios.items()):
    print("running {}".format(name))
    scen_results = pymagicc.run(scen, magicc_version=7)
    if i != 0:
        results.append(scen_results)
    else:
        results = scen_results

In [ ]:
results.filter(
    variable="Surface Temperature",
    region="World"
).line_plot(x="time");

## The faster, mildly more difficult way

Use the context manager for the MAGICC version we want. This still creates a copy of the entire run folder, but only once.

In [ ]:
with MAGICC7() as magicc:
    for i, (name, scen) in enumerate(scenarios.items()):
        # note that we obviously don't need magicc_version as an argument 
        # here as it is assigned automatically
        print("running {}".format(name))
        scen_results = magicc.run(scen)
        if i != 0:
            results.append(scen_results)
        else:
            results = scen_results
        
results.filter(
    variable="Surface Temperature",
    region="World"
).line_plot(x="time");

## The even faster, more dangerous and difficult to debug way

Use the context manager for the MAGICC version we want and specify the root directory. This doesn't create a copy of the entire run folder, but will modify files in place and hence alter any existing settings in your local copy.

In [ ]:
with MAGICC7(root_dir="../../magicc") as magicc:
    # binary readers aren't yet ready and we can't do this through
    # the run method just yet
    magicc.set_output_variables(
        write_ascii=False, write_binary=True, 
        keydata_1=True, emissions=False,
        parameters=True
    )  
    for i, (name, scen) in enumerate(scenarios.items()):
        # note that we obviously don't need magicc_version as an argument 
        # here as it is assigned automatically
        print("running {}".format(name))
        scen_results = magicc.run(scen)
        if i != 0:
            results.append(scen_results)
        else:
            results = scen_results
        
results.filter(
    variable="Surface Temperature",
    region="World"
).line_plot(x="time");

## Comparing MAGICC6 and MAGICC7

Finally, we show how to easily compare MAGICC6 and MAGICC7.

In [ ]:
for j, mclass in enumerate([MAGICC6, MAGICC7]):
    print("MAGICC{}".format(mclass.version))
    with mclass() as magicc:
        # binary readers aren't yet ready and we can't do this through
        # the run method just yet
        magicc.set_output_variables(
            write_ascii=False, write_binary=True, 
            keydata_1=True, emissions=False,
            parameters=True
        )  
        for i, (name, scen) in enumerate(scenarios.items()):
            # note that we obviously don't need magicc_version as an argument 
            # here as it is assigned automatically
            print("running {}".format(name))
            scen_results = magicc.run(scen)
            if (i != 0) or (j != 0):
                results.append(scen_results)
            else:
                results = scen_results

In [ ]:
results.filter(
    variable="Surface Temperature",
    region="World"
).line_plot(x="time");